In [1]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
X = {}
y = {}
X["train"] = np.load("X_train.npy")
y["train"] = np.load("y_train.npy")
X["dev"] = np.load("X_dev.npy")
y["dev"] = np.load("y_dev.npy")
X_text_train = np.load("X_train_text.npy")
X_text_dev = np.load("X_test_text.npy")
X["train"].shape, X["dev"].shape

((9025, 300), (1003, 300))

In [2]:
from sklearn.linear_model import LogisticRegression
l2 = 10
model = LogisticRegression(penalty='l2', C=1/l2)
model.fit(X["train"], y["train"])
model.score(X["dev"], y["dev"])

0.7666999002991027

In [3]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
gradient_train.shape, gradient_dev.shape

((9025, 301), (1003, 301))

In [4]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [5]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(1003, 9025)

In [6]:
np.save("old_predictions.npy", pred)

In [7]:
def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > 0.5:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]

    X_k = np.delete(X["train"], top_k_index, axis=0)
    y_k = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    print("prediction", prediction)

    return X_k, y_k, prediction

In [8]:
ls

CSV_emo_LR_alg1.csv     X_dev.npy               new_pred.npy
CSV_emo_LR_alg1.ipynb   X_test_text.npy         npy2tsv.ipynb
Errors_k.ipynb          X_train.npy             old_predictions.npy
Smallest_k.ipynb        X_train_text.npy        y_dev.npy
Smallest_k_valid.ipynb  appro_ks.npy            y_test.npy
Smallest_k_valid.py     emo_dev.tsv             y_test_text.npy
Untitled.ipynb          emo_train.tsv           y_train.npy
Use_approK.ipynb        hate_speech18/          y_train_text.npy


In [9]:
app_k = np.array(np.load("appro_ks.npy", allow_pickle=True).squeeze())
app_k_nonon=[]
for i in range(X["dev"].shape[0]):
    print(app_k[i])
    if (not app_k[i] > 0):
        app_k_nonon.append(X["train"].shape[0])
    else:
        app_k_nonon.append(app_k[i])

43.0
139.0
3006.0
287.0
168.0
103.0
1851.0
15.0
62.0
nan
39.0
182.0
1733.0
2395.0
1400.0
294.0
521.0
320.0
324.0
501.0
39.0
328.0
140.0
74.0
259.0
262.0
47.0
325.0
96.0
265.0
nan
230.0
111.0
51.0
nan
4505.0
2.0
29.0
nan
141.0
246.0
42.0
134.0
1520.0
380.0
1455.0
354.0
408.0
65.0
1313.0
160.0
323.0
40.0
nan
161.0
67.0
1050.0
2163.0
322.0
19.0
137.0
nan
707.0
337.0
170.0
2636.0
110.0
365.0
305.0
308.0
442.0
103.0
207.0
nan
61.0
128.0
17.0
362.0
19.0
57.0
255.0
45.0
750.0
110.0
794.0
2121.0
291.0
46.0
2993.0
390.0
356.0
704.0
89.0
421.0
8.0
42.0
954.0
1011.0
135.0
nan
21.0
60.0
132.0
325.0
39.0
95.0
165.0
499.0
893.0
3842.0
1605.0
2551.0
nan
961.0
nan
46.0
103.0
304.0
43.0
nan
742.0
82.0
236.0
23.0
356.0
61.0
333.0
8.0
97.0
1175.0
213.0
883.0
380.0
1685.0
nan
285.0
938.0
193.0
nan
37.0
1553.0
1072.0
26.0
31.0
228.0
300.0
9.0
125.0
10.0
3215.0
nan
97.0
28.0
62.0
120.0
1962.0
21.0
702.0
118.0
nan
1250.0
393.0
39.0
368.0
1050.0
208.0
472.0
1066.0
6.0
56.0
1264.0
nan
53.0
28.0
58.0
157.0
285.

In [10]:
sort_dev_k = np.array(app_k_nonon).argsort()
sort_dev_k

array([276, 741, 554, ..., 285, 493, 585])

In [11]:
app_k_nonon[sort_dev_k[0]]

1.0

In [12]:
header = ["test_movie_id", "test_movie_review_text", "old_prediction", "right_prediction", "k", "successful_flip"]
for i in range(X["train"].shape[0]):
    header.append("train_example_" + str(i+1))

In [13]:
new_predictions = np.load("new_pred.npy", allow_pickle=True)

In [14]:
data = []
for test_movie_id in sort_dev_k:
    
    test_point = []
    # id
    test_point.append(test_movie_id)
    # test_movie_review_text
    test_point.append("label: " + str(y["dev"][test_movie_id]) + "  doc: " + str(X_text_dev[test_movie_id]))
    test_point.append(pred[test_movie_id].item())
    test_point.append(np.round(pred[test_movie_id].item()) == y["dev"][test_movie_id].item())
    # k
    k = app_k[test_movie_id]
    test_point.append(str(k))
    print("k", k)
    if not (k >= 0):
        test_point.append(None)
        data.append(test_point)
        continue
    k = int(k)
    test_point.append(np.round(new_predictions[test_movie_id].item()) != np.round(pred[test_movie_id].item()))
        
    
    if pred[test_movie_id] > 0.5:
        top_k_index = delta_pred[test_movie_id].argsort()[-k:]
    else:
        top_k_index = delta_pred[test_movie_id].argsort()[:k]
    
    # training examples
    for idx in top_k_index:
        test_point.append("id: " + str(idx) + "  label: " + str(y["train"][idx]) + "  doc: " + str(X_text_train[idx]))
    data.append(test_point)

k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 2.0
k 2.0
k 2.0
k 2.0
k 2.0
k 2.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 4.0
k 4.0
k 4.0
k 4.0
k 4.0
k 4.0
k 5.0
k 5.0
k 5.0
k 5.0
k 5.0
k 6.0
k 6.0
k 6.0
k 6.0
k 7.0
k 7.0
k 8.0
k 8.0
k 8.0
k 8.0
k 8.0
k 9.0
k 9.0
k 9.0
k 9.0
k 9.0
k 9.0
k 9.0
k 9.0
k 10.0
k 10.0
k 10.0
k 10.0
k 10.0
k 10.0
k 10.0
k 11.0
k 11.0
k 11.0
k 11.0
k 11.0
k 11.0
k 12.0
k 12.0
k 12.0
k 12.0
k 12.0
k 12.0
k 13.0
k 14.0
k 15.0
k 15.0
k 15.0
k 15.0
k 15.0
k 16.0
k 16.0
k 16.0
k 16.0
k 16.0
k 16.0
k 16.0
k 16.0
k 17.0
k 17.0
k 17.0
k 17.0
k 17.0
k 17.0
k 18.0
k 18.0
k 18.0
k 19.0
k 19.0
k 19.0
k 19.0
k 20.0
k 21.0
k 21.0
k 21.0
k 21.0
k 21.0
k 22.0
k 22.0
k 22.0
k 22.0
k 22.0
k 22.0
k 23.0
k 23.0
k 24.0
k 24.0
k 24.0
k 25.0
k 25.0
k 26.0
k 26.0
k 26.0
k 27.0
k 27.0
k 27.0
k 28.0
k 28.0
k 28.0
k 29.0
k 29.0
k 30.0
k 31.0
k 31.0
k 31.0
k 31.0
k 32.0
k 32.0
k 32.0
k 33.0
k 33.0
k 33.0
k 33.0
k 33.0
k 34.0
k 34.0
k 35.0
k 36.0
k 36.0
k 36.0
k 36.0
k 36.0
k 37.0
k 37.0


In [15]:
import csv

with open('CSV_emo_LR_alg1.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)


In [16]:
ls

CSV_emo_LR_alg1.csv     X_dev.npy               new_pred.npy
CSV_emo_LR_alg1.ipynb   X_test_text.npy         npy2tsv.ipynb
Errors_k.ipynb          X_train.npy             old_predictions.npy
Smallest_k.ipynb        X_train_text.npy        y_dev.npy
Smallest_k_valid.ipynb  appro_ks.npy            y_test.npy
Smallest_k_valid.py     emo_dev.tsv             y_test_text.npy
Untitled.ipynb          emo_train.tsv           y_train.npy
Use_approK.ipynb        hate_speech18/          y_train_text.npy


In [44]:
pwd

'/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_SST/IF_lr_fix'